# Predicting preferred destination  based on taste and preference

The goal is to build a machine learning model that can predict hotel ratings based on customer reviews, budget, specific locations, and the type of residence. The dataset is scraped from TripAdvisor and it contains information about various hotels, including their ratings, reviews, amenities, pricing, geographical coordinates, and residence types (e.g., hotel, bed and breakfast, specialty lodging). By analyzing the text reviews along with these additional factors, the objective is to develop a model that can accurately predict the ratings of new, unseen hotels based on customer reviews, budget constraints, location preferences, and residence type.

Approach:

Data Preprocessing: Clean and preprocess the text reviews by removing stopwords, punctuation, and performing tokenization. Convert the text data into a numerical representation suitable for modeling. Handle missing values, if any, in the budget, location, and residence type columns.

Feature Engineering: Extract additional features from the dataset, such as review sentiment scores, review length, and any other relevant information. Engineer new features related to budget, location, and residence type, such as price range categories, geographical distance from landmarks, and one-hot encoding of residence types.

Model Selection: Experiment with different supervised learning models, such as linear regression, decision trees, random forests, or neural networks, to find the best model for predicting hotel ratings considering customer reviews, budget, location, and residence type. Evaluate the models using appropriate evaluation metrics like mean squared error (MSE) or mean absolute error (MAE).

Model Training and Evaluation: Split the dataset into training and testing sets. Train the selected model on the training set and evaluate its performance on the testing set. Fine-tune the model parameters to improve its accuracy. Perform cross-validation to assess the model's generalization capabilities.

In [1]:
import pandas as pd
import json
import glob
import re

In [2]:
def read_json_files(json_files):
    dfs = []
    for file in json_files:
        with open(file) as f:
            json_data = json.load(f)
            df = pd.DataFrame(json_data)
            dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)
    return merged_df



In [3]:
json_files = ['travel-destination-recommendation-sys\Data\drc.json','travel-destination-recommendation-sys\Data\egypt.json', 'travel-destination-recommendation-sys\Data\ethiopia.json',
                'travel-destination-recommendation-sys\Data\kenya.json', 'travel-destination-recommendation-sys\Data\Madagascar.json', 'travel-destination-recommendation-sys\Data\morocco.json',
                r'travel-destination-recommendation-sys\Data\nigeria.json', r'travel-destination-recommendation-sys\Data\rwanda.json', 'travel-destination-recommendation-sys\Data\seychelles.json',
                r'travel-destination-recommendation-sys\Data\tanzania.json', r'travel-destination-recommendation-sys\Data\uganda.json', r'travel-destination-recommendation-sys\Data\namibia.json',
                'travel-destination-recommendation-sys\Data\SouthAfrica.json', 'travel-destination-recommendation-sys\Data\malawi.json', 'travel-destination-recommendation-sys\Data\Senegal.json',
                'travel-destination-recommendation-sys\Data\zambia.json', 'travel-destination-recommendation-sys\Data\Ghana.json', 'travel-destination-recommendation-sys\Data\Botswana.json', 
                'travel-destination-recommendation-sys\Data\capeverde.json' ]
df = read_json_files(json_files)



In [4]:
df

,id,type,category,subcategories,name,locationString,description,image,photoCount,awards,...,establishmentTypes,ownersTopReasons,localLangCode,guideFeaturedInCopy,rentalDescriptions,photos,bedroomInfo,bathroomInfo,bathCount,baseDailyRate
0,2704993,ATTRACTION,attraction,[Nature & Parks],Congoloisirs,Kinshasa,None,https://media-cdn.tripadvisor.com/media/photo-...,9,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1536776,ATTRACTION,attraction,[Nature & Parks],Okapi Wildlife Reserve,Orientale Province,None,https://media-cdn.tripadvisor.com/media/photo-...,3,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13203729,ATTRACTION,attraction,"[Shopping, Food & Drink]",Marche Nouveau DAIPN,Kinshasa,None,https://media-cdn.tripadvisor.com/media/photo-...,12,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8661504,HOTEL,hotel,[Specialty Lodging],Bukima Tented Camp,"Rumangabo, North Kivu Province",Just outside the Virunga National Park boundar...,https://media-cdn.tripadvisor.com/media/photo-...,79,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10414108,HOTEL,hotel,[Specialty Lodging],"Tchegera Island Tented Camp, Virunga National ...","Goma, North Kivu Province",None,https://media-cdn.tripadvisor.com/media/photo-...,109,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35831,12216827,HOTEL,hotel,[Specialty Lodging],Casa Santos Pinto,"Curral das Vacas, Santo Antao",None,None,0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35832,23200009,HOTEL,hotel,[Bed and Breakfast],Kelly GuestHouse - Lovely Bedroom - Plateau Ci...,"Praia, Santiago",None,https://media-cdn.tripadvisor.com/media/partne...,0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35833,13423426,HOTEL,hotel,[Bed and Breakfast],Luz Esperanca,"Pedra Badejo, Santiago",None,None,0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35834,12957229,HOTEL,hotel,[Specialty Lodging],Pensao Entre Nos,"Tarrafal, Santiago",None,None,0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.isnull().sum()

id                   0
type                 0
category             0
subcategories     1339
name                 0
                 ...  
photos           34497
bedroomInfo      34497
bathroomInfo     34497
bathCount        34497
baseDailyRate    34568
Length: 65, dtype: int64

In [6]:
df.columns

Index(['id', 'type', 'category', 'subcategories', 'name', 'locationString',
       'description', 'image', 'photoCount', 'awards', 'rankingPosition',
       'rating', 'rawRanking', 'phone', 'address', 'addressObj', 'localName',
       'localAddress', 'email', 'latitude', 'longitude', 'webUrl', 'website',
       'rankingString', 'rankingDenominator', 'neighborhoodLocations',
       'nearestMetroStations', 'ancestorLocations', 'ratingHistogram',
       'numberOfReviews', 'reviewTags', 'reviews', 'booking', 'offerGroup',
       'subtype', 'hotelClass', 'amenities', 'numberOfRooms', 'priceLevel',
       'priceRange', 'roomTips', 'checkInDate', 'checkOutDate', 'offers',
       'hotelClassAttribution', 'isClosed', 'isLongClosed', 'openNowText',
       'cuisines', 'mealTypes', 'dishes', 'features', 'dietaryRestrictions',
       'hours', 'menuWebUrl', 'establishmentTypes', 'ownersTopReasons',
       'localLangCode', 'guideFeaturedInCopy', 'rentalDescriptions', 'photos',
       'bedroomInfo', '

In [7]:
df.reviewTags.value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [8]:
# Check null values and filter columns with more than 4000 null values
null_counts = df.isnull().sum()
columns_above_threshold = null_counts[null_counts > 10000].index

# Print the columns with more than 4000 null values
list(columns_above_threshold)


['description',
 'phone',
 'localAddress',
 'email',
 'website',
 'booking',
 'offerGroup',
 'subtype',
 'hotelClass',
 'numberOfRooms',
 'priceLevel',
 'priceRange',
 'roomTips',
 'checkInDate',
 'checkOutDate',
 'offers',
 'hotelClassAttribution',
 'isClosed',
 'isLongClosed',
 'openNowText',
 'cuisines',
 'mealTypes',
 'dishes',
 'features',
 'dietaryRestrictions',
 'hours',
 'menuWebUrl',
 'establishmentTypes',
 'ownersTopReasons',
 'localLangCode',
 'guideFeaturedInCopy',
 'rentalDescriptions',
 'photos',
 'bedroomInfo',
 'bathroomInfo',
 'bathCount',
 'baseDailyRate']

In [9]:
# we will drop the following columns because they do not have any contribution to our objectives.
# some also contain too many null values to fill. 
cols_to_drop = columns_above_threshold

df.drop(columns=cols_to_drop, inplace=True)

In [10]:
list(df.columns)

['id',
 'type',
 'category',
 'subcategories',
 'name',
 'locationString',
 'image',
 'photoCount',
 'awards',
 'rankingPosition',
 'rating',
 'rawRanking',
 'address',
 'addressObj',
 'localName',
 'latitude',
 'longitude',
 'webUrl',
 'rankingString',
 'rankingDenominator',
 'neighborhoodLocations',
 'nearestMetroStations',
 'ancestorLocations',
 'ratingHistogram',
 'numberOfReviews',
 'reviewTags',
 'reviews',
 'amenities']

In [11]:
df[['locationString','rankingPosition','rawRanking','rankingString','rankingDenominator']]

,locationString,rankingPosition,rawRanking,rankingString,rankingDenominator
0,Kinshasa,17.0,2.778074,#17 of 105 things to do in Kinshasa,105
1,Orientale Province,1.0,2.751658,#1 of 4 things to do in Orientale Province,4
2,Kinshasa,21.0,2.773659,#21 of 105 things to do in Kinshasa,105
3,"Rumangabo, North Kivu Province",2.0,3.351389,#2 of 3 Specialty lodging in Rumangabo,3
4,"Goma, North Kivu Province",1.0,3.464931,#1 of 17 Specialty lodging in Goma,17
...,...,...,...,...,...
35831,"Curral das Vacas, Santo Antao",NaN,NaN,None,None
35832,"Praia, Santiago",NaN,NaN,None,None
35833,"Pedra Badejo, Santiago",NaN,NaN,None,None
35834,"Tarrafal, Santiago",NaN,NaN,None,None


In [12]:
df[['name','rankingString']]

,name,rankingString
0,Congoloisirs,#17 of 105 things to do in Kinshasa
1,Okapi Wildlife Reserve,#1 of 4 things to do in Orientale Province
2,Marche Nouveau DAIPN,#21 of 105 things to do in Kinshasa
3,Bukima Tented Camp,#2 of 3 Specialty lodging in Rumangabo
4,"Tchegera Island Tented Camp, Virunga National ...",#1 of 17 Specialty lodging in Goma
...,...,...
35831,Casa Santos Pinto,None
35832,Kelly GuestHouse - Lovely Bedroom - Plateau Ci...,None
35833,Luz Esperanca,None
35834,Pensao Entre Nos,None


In [13]:


# Assuming your data is in a DataFrame called 'df' and the column is named 'rankingString'
# Create new columns
df['RankingType'] = ""
df['Location'] = ""
df['Numerator'] = ""
df['Denominator'] = ""

# Iterate through the rows and extract the information
for index, row in df.iterrows():
    # Check if the value is NaN
    if pd.isnull(row['rankingString']):
        continue
    
    # Use regular expressions to extract the information
    match = re.match(r'#(\d+)\s+of\s+(\d+)\s+(.*?)\s+in\s+(.*?)$', row['rankingString'])
    
    # Check if the match is successful
    if match:
        numerator = match.group(1)
        denominator = match.group(2)
        ranking_type = match.group(3)
        location = match.group(4)
        
        # Update the new columns
        df.at[index, 'RankingType'] = ranking_type
        df.at[index, 'Location'] = location
        df.at[index, 'Numerator'] = numerator
        df.at[index, 'Denominator'] = denominator



In [78]:
df.columns

Index(['id', 'type', 'category', 'subcategories', 'name', 'locationString',
       'image', 'photoCount', 'awards', 'rankingPosition', 'rating',
       'rawRanking', 'address', 'addressObj', 'localName', 'latitude',
       'longitude', 'webUrl', 'rankingString', 'rankingDenominator',
       'neighborhoodLocations', 'nearestMetroStations', 'ancestorLocations',
       'ratingHistogram', 'numberOfReviews', 'reviewTags', 'reviews',
       'amenities', 'RankingType', 'Location', 'Numerator', 'Denominator'],
      dtype='object')

In [86]:
df.RankingType.value_counts()

                             9930
Specialty lodging            7287
B&Bs / Inns                  6045
hotels                       4718
things to do                 3263
Outdoor Activities           1298
Tours                         693
Boat Tours & Water Sports     558
Transportation                532
places to eat                 326
hotel                         243
B&B / Inn                     239
Shopping                      162
Food & Drink                  161
Nightlife                     126
Spas & Wellness               115
Fun & Games                    73
Classes & Workshops            37
Nature & Parks                 12
Museums                         8
Concerts & Shows                7
Traveler Resources              1
Water & Amusement Parks         1
Sights & Landmarks              1
Name: RankingType, dtype: int64

In [ ]:
# Define the mappings to combine similar values
mappings = {
    'hotel': 'hotels',
    'B&B / Inn': 'B&Bs / Inns',
    
}

# Replace the values in the 'Ranking Type' column
df['Ranking Type'] = df['Ranking Type'].replace(mappings)

In [18]:
df.amenities

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                              [Restaurant, Mountain View]
4        [Multilingual Staff, Restaurant, Bar/Lounge, F...
                               ...                        
35831    [Shuttle Bus Service, Restaurant, Bar/Lounge, ...
35832                                                   []
35833    [Kids Activities, Free parking, Airport transp...
35834    [Kids Activities, Free parking, Airport transp...
35835                                                   []
Name: amenities, Length: 35836, dtype: object

In [26]:
null_values = df[df['amenities'].isnull()]
null_values

,id,type,category,subcategories,name,locationString,image,photoCount,awards,rankingPosition,...,ancestorLocations,ratingHistogram,numberOfReviews,reviewTags,reviews,amenities,RankingType,Location,Numerator,Denominator
0,2704993,ATTRACTION,attraction,[Nature & Parks],Congoloisirs,Kinshasa,https://media-cdn.tripadvisor.com/media/photo-...,9,[],17.0,...,"[{'id': '294187', 'name': 'Kinshasa', 'abbrevi...","{'count1': 0, 'count2': 0, 'count3': 4, 'count...",9,[],[],NaN,things to do,Kinshasa,17,105
1,1536776,ATTRACTION,attraction,[Nature & Parks],Okapi Wildlife Reserve,Orientale Province,https://media-cdn.tripadvisor.com/media/photo-...,3,[],1.0,...,"[{'id': '1536771', 'name': 'Orientale Province...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",2,[],[],NaN,things to do,Orientale Province,1,4
2,13203729,ATTRACTION,attraction,"[Shopping, Food & Drink]",Marche Nouveau DAIPN,Kinshasa,https://media-cdn.tripadvisor.com/media/photo-...,12,[],21.0,...,"[{'id': '294187', 'name': 'Kinshasa', 'abbrevi...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",3,[],[],NaN,things to do,Kinshasa,21,105
6,19492774,ATTRACTION,attraction,[Outdoor Activities],Cercle Elais,Kinshasa,https://media-cdn.tripadvisor.com/media/photo-...,8,[],14.0,...,"[{'id': '294187', 'name': 'Kinshasa', 'abbrevi...","{'count1': 1, 'count2': 0, 'count3': 0, 'count...",6,"[{'text': 'pool', 'reviews': 2}]",[],NaN,things to do,Kinshasa,14,105
7,4889528,ATTRACTION,attraction,[Sights & Landmarks],Eglise CBFC-Gombe,Kinshasa,https://media-cdn.tripadvisor.com/media/photo-...,13,[],9.0,...,"[{'id': '294187', 'name': 'Kinshasa', 'abbrevi...","{'count1': 0, 'count2': 0, 'count3': 1, 'count...",11,[],[],NaN,things to do,Kinshasa,9,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35299,25810865,ATTRACTION,attraction,[Spas & Wellness],"Renova Spa, RIU Palace Boa Vista","Rabil, Boa Vista",https://media-cdn.tripadvisor.com/media/photo-...,7,[],NaN,...,"[{'id': '13867343', 'name': 'Rabil', 'abbrevia...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",0,[],[],NaN,,,,
35301,17504922,ATTRACTION,attraction,"[Tours, Other, Boat Tours & Water Sports, Tran...",Sea Turtle Sal,"Santa Maria, Ilha do Sal",https://media-cdn.tripadvisor.com/media/photo-...,6,[],13.0,...,"[{'id': '482848', 'name': 'Santa Maria', 'abbr...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",2,[],[],NaN,Transportation,Santa Maria,13,27
35305,25416857,ATTRACTION,attraction,[Tours],Sidy Tours,"Sal Rei, Boa Vista",None,0,[],NaN,...,"[{'id': '1185333', 'name': 'Sal Rei', 'abbrevi...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",0,[],[],NaN,,,,
35307,20282100,ATTRACTION,attraction,"[Tours, Other, Transportation, Outdoor Activit...","Over Clauds Tours- Servico de Guia, Lda.","Mindelo, Sao Vicente",https://media-cdn.tripadvisor.com/media/photo-...,1,[],NaN,...,"[{'id': '482855', 'name': 'Mindelo', 'abbrevia...","{'count1': 0, 'count2': 0, 'count3': 0, 'count...",0,[],[],NaN,,,,


In [24]:
df[['name', 'amenities']].groupby('amenities').value_counts()

TypeError: unhashable type: 'list'

In [22]:
# data.to_csv(r"E:\Documents\data science\Capstone\data1")

In [23]:
from pandas_profiling import ProfileReport

C:\Users\BobbyBob\AppData\Local\Temp\ipykernel_16380\2274191625.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [6]:
import pandas_profiling


C:\Users\BobbyBob\AppData\Local\Temp\ipykernel_8796\3886088315.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [7]:
profile_trip = pandas_profiling.ProfileReport(df)
profile_trip.to_file("df.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\BobbyBob\anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:67: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
